# Deep Learning Project \#1

In [1]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt

Downloading the dataset:

In [2]:
trainingdata = torchvision.datasets.CIFAR10('./CIFAR-10/',train=True,download=True)
testdata = torchvision.datasets.CIFAR10('./CIFAR-10/',train=False,download=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./CIFAR-10/cifar-10-python.tar.gz to ./CIFAR-10/
Files already downloaded and verified


Mean and Std calculation for normalization:

In [3]:
tr_mean = trainingdata.data.mean(axis = (0,1,2)) / 255
tr_std = trainingdata.data.std(axis = (0,1,2))  / 255
print(tr_mean)
print(tr_std)

[0.49139968 0.48215841 0.44653091]
[0.24703223 0.24348513 0.26158784]


Data Augmentation (Random Transforms and Normalization)

In [4]:
tr_train = torchvision.transforms.Compose([
        torchvision.transforms.RandomCrop(size=[32,32], padding=4),
        torchvision.transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        torchvision.transforms.RandomRotation(10),
        torchvision.transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(tr_mean, tr_std)
    ])

tr_test = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                          torchvision.transforms.Normalize(tr_mean, tr_std)])

Download the datasets

In [5]:
trainingdata = torchvision.datasets.CIFAR10('./CIFAR-10/',train=True,download=False,transform=tr_train)
testdata = torchvision.datasets.CIFAR10('./CIFAR-10/',train=False,download=False,transform=tr_test)

Check how many Training and Test Data we have

In [6]:
print(len(trainingdata))
print(len(testdata))

50000
10000


Printing size of the images:

In [7]:
image, label = trainingdata[13]
print(image.size(),label)

torch.Size([3, 32, 32]) 2


We set up our data loaders.  
We can try different Batch Sizes here.

In [8]:
trainDataLoader = torch.utils.data.DataLoader(trainingdata,batch_size=64,shuffle=True)
testDataLoader = torch.utils.data.DataLoader(testdata,batch_size=64,shuffle=False)

### Generic ResNet Model:

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):

    def __init__(self, in_planes, planes, kernel_size, skip_kernel_size, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size, stride=stride, padding=int((kernel_size-1)/2), bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size,
                               stride=1, padding=int((kernel_size-1)/2), bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes,
                          skip_kernel_size, stride=stride, padding=int((skip_kernel_size-1)/2), bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out



class ResNet(nn.Module):
    def __init__(self, block, N, num_blocks, C_1, conv_kernel_size, skip_kernel_size, avg_pool_size, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = C_1
        self.avg_pool_size = avg_pool_size

        self.conv1 = nn.Conv2d(3, C_1, conv_kernel_size[0],
                               stride=1, padding=int((conv_kernel_size[0]-1)/2), bias=False)
        self.bn1 = nn.BatchNorm2d(C_1)

        res_layers = []
        
        for i in range(N):
          if i == 0:
            s = 1
          else:
            s = 2
          res_layers.append(self._make_layer(block, (2**i)*C_1, num_blocks[i], conv_kernel_size[i], skip_kernel_size[i], stride=s))

        self.layer = nn.Sequential(*res_layers)

        ps = 2**(np.ceil(np.log2(avg_pool_size)))


        self.linear = nn.Linear(int(C_1*32*32/(2**(N-1))/(ps**2)), num_classes)

    def _make_layer(self, block, planes, num_blocks, conv_kernel_size, skip_kernel_size, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, conv_kernel_size, skip_kernel_size, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer(out)
        out = F.avg_pool2d(out, self.avg_pool_size)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def project1_model(N, num_blocks, C_1, conv_kernel_size, skip_kernel_size, avg_pool_size):
    return ResNet(BasicBlock, N, num_blocks, C_1, conv_kernel_size, skip_kernel_size, avg_pool_size, num_classes=10)





### We defined our model in a parametric way

So here we can set up our parameters.

N: Number of residual layers, 3-4-5  
C_1: # channels in the first layer  
num_blocks: Nx1 vector, num_blocks[i] is number of residual blocks in layer i  
conv_kernel_size: Nx1 vector, kernel size for conv layers in each residual layer  
skip_kernel_size: Nx1 vector, kernel size for skip connections between each residual block  
avg_pool_size: Kernel size for the average pooling at last layer  

In [19]:
N = 3
C_1 = 85
num_blocks = [2]*N
conv_kernel_size = [3]*N
avg_pool_size = 8
skip_kernel_size = [1]*N

lr = 0.001

In [20]:
net = project1_model(N,
                    num_blocks, 
                    C_1, 
                    conv_kernel_size,
                    skip_kernel_size,
                    avg_pool_size).cuda()
Loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)

### Summary

In [21]:
# Summary of the model
from torchinfo import summary
from torchvision import models

summary(net,(1,3,32,32))

Layer (type:depth-idx)                        Output Shape              Param #
ResNet                                        --                        --
├─Conv2d: 1-1                                 [1, 85, 32, 32]           2,295
├─BatchNorm2d: 1-2                            [1, 85, 32, 32]           170
├─Sequential: 1-3                             [1, 340, 8, 8]            --
│    └─Sequential: 2-1                        [1, 85, 32, 32]           --
│    │    └─BasicBlock: 3-1                   [1, 85, 32, 32]           130,390
│    │    └─BasicBlock: 3-2                   [1, 85, 32, 32]           130,390
│    └─Sequential: 2-2                        [1, 170, 16, 16]          --
│    │    └─BasicBlock: 3-3                   [1, 170, 16, 16]          405,620
│    │    └─BasicBlock: 3-4                   [1, 170, 16, 16]          520,880
│    └─Sequential: 2-3                        [1, 340, 8, 8]            --
│    │    └─BasicBlock: 3-5                   [1, 340, 8, 8]           

### Training

In [13]:
import time

def neural_net(N,C1,num_blocks,conv_kernel_size,avg_pool_size,skip_kernel_size):
    net = project1_model(N,
                    num_blocks, 
                    C1, 
                    conv_kernel_size,
                    skip_kernel_size,
                    avg_pool_size).cuda()
    return net

In [14]:
Grid_Search = ((2,1,256), #1
              (2,2,172), #2
              (2,3,140), #3
              (2,4,120), #4
              (2,5,106), #5
              (2,6,96),  #6
              (2,7,90),  #7
              (3,1,128), #8
              (3,2,85),  #9
              (3,3,68),  #10
              (3,4,58),  #11
              (3,5,52),  #12
              (4,1,64),  #13
              (4,2,42),  #14
              (4,3,32),  #15
              (4,4,29),  #16
              (4,5,26),  #17
              (5,1,32),  #18
              (5,2,21),  #19
              (5,3,17),  #20
              (5,4,14),  #21
              (5,5,13))  #22

Num_Runs = len(Grid_Search)
Num_Epochs = 50

In [15]:
train_loss_history = np.zeros([Num_Runs, Num_Epochs])
test_loss_history = np.zeros([Num_Runs, Num_Epochs])
train_acc_history = np.zeros([Num_Runs, Num_Epochs])
test_acc_history = np.zeros([Num_Runs, Num_Epochs])

# START EDITING FROM HERE

In [16]:
for i in range(0,5):
    T1 = time.time()
    print(f'Run Number: {i+1} Starts Here')
    N,B,C = Grid_Search[i]
    num_blocks = [B]*N
    skip_kernel_size = [1]*N
    conv_kernel_size = [3]*N
    avg_pool_size = int(32/(2**(N-1)))

    net = neural_net(N,C,num_blocks,conv_kernel_size,avg_pool_size,skip_kernel_size)

    Loss = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)

    for epoch in range(Num_Epochs):
        T11 = time.time()

        train_loss = 0.0
        test_loss = 0.0

        train_acc = 0.0
        test_acc = 0.0

        for k, data in enumerate(trainDataLoader):
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            optimizer.zero_grad()
            predicted_output = net(images)
            fit = Loss(predicted_output,labels)
            fit.backward()
            optimizer.step()
            train_loss += fit.item()
            train_acc += torch.sum(labels == predicted_output.argmax(dim=1)).item()

        for k, data in enumerate(testDataLoader):
            with torch.no_grad():
                images, labels = data
                images = images.cuda()
                labels = labels.cuda()
                predicted_output = net(images)
                fit = Loss(predicted_output,labels)
                test_loss += fit.item()
                test_acc += torch.sum(labels == predicted_output.argmax(dim=1)).item()

        scheduler.step()
        train_loss = train_loss/len(trainDataLoader)
        test_loss = test_loss/len(testDataLoader)
        train_acc = train_acc/len(trainingdata)
        test_acc = test_acc/len(testdata)
        
        train_acc_history[i][epoch] = train_acc 
        test_acc_history[i][epoch] = test_acc
        train_loss_history[i][epoch] = train_loss
        test_loss_history[i][epoch] = test_loss
        
        T22 = time.time()

        print(f'Epoch: {epoch} \tTrain Loss: {train_loss:.5f} \tTrain Acc: {train_acc:.4f} \tTest Loss: {test_loss:.5f} \tTest Acc: {test_acc:.4f} \tTime: {T22-T11:.4f}')

    T2 = time.time()
    
    print(f'Total Time For This Run: {T2-T1:.4f}')
        

Run Number: 1 Starts Here
Epoch: 0 	Train Loss: 1.61690 	Train Acc: 0.3988 	Test Loss: 1.43155 	Test Acc: 0.4908 	Time: 132.7520
Epoch: 1 	Train Loss: 1.29677 	Train Acc: 0.5353 	Test Loss: 1.32635 	Test Acc: 0.5383 	Time: 132.3871
Epoch: 2 	Train Loss: 1.15550 	Train Acc: 0.5913 	Test Loss: 1.13582 	Test Acc: 0.6066 	Time: 132.3256
Epoch: 3 	Train Loss: 1.07145 	Train Acc: 0.6204 	Test Loss: 1.15049 	Test Acc: 0.6104 	Time: 132.1676
Epoch: 4 	Train Loss: 0.98373 	Train Acc: 0.6556 	Test Loss: 0.96248 	Test Acc: 0.6645 	Time: 132.0584
Epoch: 5 	Train Loss: 0.90878 	Train Acc: 0.6825 	Test Loss: 0.91594 	Test Acc: 0.6867 	Time: 132.1401
Epoch: 6 	Train Loss: 0.85890 	Train Acc: 0.7001 	Test Loss: 0.81027 	Test Acc: 0.7192 	Time: 132.0490
Epoch: 7 	Train Loss: 0.80593 	Train Acc: 0.7204 	Test Loss: 0.80606 	Test Acc: 0.7276 	Time: 132.0495
Epoch: 8 	Train Loss: 0.76922 	Train Acc: 0.7334 	Test Loss: 0.81162 	Test Acc: 0.7216 	Time: 131.6814
Epoch: 9 	Train Loss: 0.72773 	Train Acc: 0.748

In [17]:
np.savetxt('Train_Acc_0_2.txt',train_acc_history)
np.savetxt('Test_Acc_0_2.txt',test_acc_history)
np.savetxt('Train_Loss_0_2.txt',train_loss_history)
np.savetxt('Test_Loss_0_2.txt',test_loss_history)

In [18]:
import pandas as pd

df = pd.DataFrame (train_acc_history)
filepath = 'Train_Acc_0_5.xlsx'
df.to_excel(filepath, index=False)

df = pd.DataFrame (test_acc_history)
filepath = 'Test_Acc_0_5.xlsx'
df.to_excel(filepath, index=False)


df = pd.DataFrame (train_loss_history)
filepath = 'Train_Loss_0_5.xlsx'
df.to_excel(filepath, index=False)


df = pd.DataFrame (test_loss_history)
filepath = 'Test_Loss_0_5.xlsx'
df.to_excel(filepath, index=False)